# Analyzing NYC High School Data for Efficacy of Standardized Tests

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/NYC_DOE_Logo.png/1200px-NYC_DOE_Logo.png" width="400" height="400" />

*Source*: [wikipedia.org](https://en.wikipedia.org/wiki/New_York_City_Department_of_Education)

In this project, we'll be using data about New York City public schools, which can be found [here](https://data.cityofnewyork.us/browse?category=Education).

One of the most controversial issues in the US educational system is the efficacy of standardized tests and whether they're unfair to certain groups. Given our prior knowledge of this topic, investigating the correlations between [SAT scores](https://en.wikipedia.org/wiki/SAT) and demographics might be an interesting angle to take. We could correlate SAT scores with factors like race, gender, income, and more.

## Project Overview

The SAT (Scholastic Aptitude Test), is an exam that US high school students take before applying to college. Colleges take the test scores into account when deciding who to admit, so it's important to perform well.

The test consists of three sections, each of which has 800 possible points. The combined score is out of 2,400 possible (while this number has changed a few times, the dataset for our project is based on 2,400 total points). Organizations often rank high schools by their average SAT scores. The scores are also considered a measure of overall school district quality.

New York City makes its [data on high school SAT scores](https://data.cityofnewyork.us/Education/SAT-Results/f9bf-2cp4) available online, as well as the [demographic for each high school](https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2). This first few rows of the SAT data look like this:

![](https://s3.amazonaws.com/dq-content/sat.png)

*source*: [Dataquest](https://www.dataquest.io/new-home/)

## Data Sources

Unfortunately, combining both of the datasets won't give us all of the demographic information we want to use. We'll need to supplement our data with other sources to do our full analysis.

The same website has several related datasets covering demographic information and test scores. Here are the links to all of the datasets we'll be using:

- [SAT scores by school](https://data.cityofnewyork.us/Education/SAT-Results/f9bf-2cp4) - SAT scores for each high school in New York City
- [School attendance](https://data.cityofnewyork.us/Education/School-Attendance-and-Enrollment-Statistics-by-Dis/7z8d-msnt) - Attendance information for each school in New York City
- [Class size](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3) - Information on class size for each school
- [AP test results](https://data.cityofnewyork.us/Education/AP-College-Board-2010-School-Level-Results/itfs-ms3e) - Advanced Placement (AP) exam results for each high school (passing an optional AP exam in a particular subject can earn a student college credit in that subject)
- [Graduation outcomes](https://data.cityofnewyork.us/Education/Graduation-Outcomes-Classes-Of-2005-2010-School-Le/vh2h-md7a) - The percentage of students who graduated, and other outcome information
- [Demographics](https://data.cityofnewyork.us/Education/School-Demographics-and-Accountability-Snapshot-20/ihfw-zy9j) - Demographic information for each school
- [School survey](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8) - Surveys of parents, teachers, and students at each school

All of these datasets are interrelated. We'll need to combine them into a single dataset before we can find correlations.

## Background Research on Topic

Before we move into coding, we'll need to do a thorough research to understand the data. This can help us avoid costly mistakes, such as thinking that a column represents something other than what it does. Background research gives us a better understanding of how to combine and analyze the data.

In our case, we'll want to research:

- [New York City](https://en.wikipedia.org/wiki/New_York_City)
- [The SAT](https://en.wikipedia.org/wiki/SAT)
- [Schools in New York City](https://en.wikipedia.org/wiki/List_of_high_schools_in_New_York_City)
- [Our data](https://data.cityofnewyork.us/browse?category=Education)

We can learn a few different things from these resources, For example:

- Only high school students take the SAT, so we'll want to focus on high schools.
- New York City is made up of five boroughs, which are essentially distinct regions.
- New York City schools fall within several different school districts, each of which can contain dozens of schools.
- Our datasets include several different types of schools. We'll need to clean them so that we can focus on high schools only.
- Each school in New York City has a unique code called a `DBN` or district borough number.
- Aggregating data by district allows us to use the district mapping data to plot district-by-district differences.

## Import Libraries and Read in Datasets

In our first step, we'll import the libraries that are useful to work with the datasets.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

%matplotlib inline

We've imported the libraries and now we're ready to read in the data. For the convenience, we've placed all the data into the `schools` folder. Here are all of the files in the folder:

- `ap_2010.csv` - Data on [AP test results](https://data.cityofnewyork.us/Education/AP-College-Board-2010-School-Level-Results/itfs-ms3e)
- `class_size.csv` - Data on [class size](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3)
- `demographics.csv` - Data on [demographics](https://data.cityofnewyork.us/Education/School-Demographics-and-Accountability-Snapshot-20/ihfw-zy9j)
- `graduation.csv` - Data on [graduation outcomes](https://data.cityofnewyork.us/Education/Graduation-Outcomes-Classes-Of-2005-2010-School-Le/vh2h-md7a)
- `hs_directory.csv` - A directory of [high schools](https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2)
- `sat_results.csv` - Data on [SAT scores](https://data.cityofnewyork.us/Education/SAT-Results/f9bf-2cp4)
- `survey_all.txt` - Data on [surveys](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8) from all schools
- `survey_d75.txt` - Data on [surveys](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8) from New York City [district 75](https://www.schools.nyc.gov/learning/special-education/school-settings/district-75) (note: the url no longer working)

`survey_all.txt` and `survey_d75.txt` are in more complicated formats than the other files. For now, we'll focus on reading in the `CSV` files only, and then explore them.

We'll read each file into a [pandas.dataframe](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) and then store all of the dataframes in a dictionary. This gives us a convenient way to store them and a quick way to reference them later on.

In [2]:
# Create a list 'data_files' to store all files
data_files = ['ap_2010.csv', 'class_size.csv',
              'demographics.csv', 'graduation.csv',
              'hs_directory.csv', 'sat_results.csv']

# Create a dictionary 'data' to store filename as the key
# and the corresponding dataframe as its value
data = {}
for file in data_files:
    # extract file name
    filename = file.replace('.csv', '')
    # read file as pandas dataframe
    df = pd.read_csv(f'schools/{file}')
    data[filename] = df

What we're mainly interested in is the SAT dataset, which corresponds to the directory key `sat_results`. This dataset contains the SAT scores for each high school in New York City. We eventualy want to correlate selected information from this dataset with information in the other datasets.

Let's explore `sat_results` to see what we can discover. Exploring the dataframe helps us understand the structure of the data and make it easier for us to analyze it.

In [3]:
# Display the first five rows of the 'SAT scores' data
data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


We can make a few observations based on the output above:

- The `DBN` appears to be a unique ID for each school.
- We can tell from the first few rows of names that we only have data about high schools.
- There's only a single row for each high school, so each `DBN` is unique in the SAT data.
- We may eventually want to combine the three columns that contian SAT scores -- `SAT Critical Reading Avg. Score`, `SAT Math Avg. Score`, and `SAT Writing Avg. Score` -- into a single column to make the scores easier to analyze.

Given these observations, let's explore the other datasets to see if we can gain any insight into how to combine them.

In [4]:
# Display the first five rows of each dataframe in the 'data' dictionary
for k in data:
    print(data[k].head())

      DBN                             SchoolName AP Test Takers   \
0  01M448           UNIVERSITY NEIGHBORHOOD H.S.              39   
1  01M450                 EAST SIDE COMMUNITY HS              19   
2  01M515                    LOWER EASTSIDE PREP              24   
3  01M539         NEW EXPLORATIONS SCI,TECH,MATH             255   
4  02M296  High School of Hospitality Management               s   

  Total Exams Taken Number of Exams with scores 3 4 or 5  
0                49                                   10  
1                21                                    s  
2                26                                   24  
3               377                                  191  
4                 s                                    s  
   CSD BOROUGH SCHOOL CODE                SCHOOL NAME GRADE  PROGRAM TYPE  \
0    1       M        M015  P.S. 015 Roberto Clemente     0K       GEN ED   
1    1       M        M015  P.S. 015 Roberto Clemente     0K          CTT   
2    1

With these group of dataframes, we can make some observations based on the first few rows of each one.

- Each dataset appears to either have a `DBN` column or the information we need to create one. That means we can use a `DBN` column to combine the datasets. First we'll pinpoint matching rows from different datasets by looking for identical `DBN`s, then group all of their columns together in a single dataset.
- Some fields look interesting for mapping -- particularly `Location 1`, which contains coordinates inside a larger string.
- Some of the datasets appear to contain multiple rows for each school (because the rows have duplicate `DBN` values). That means we'll have to do some preprocessing to ensure that each `DBN` is unique within each dataset. If we don't do this, we'll run into problems when we combine the datasets, because we might be merging two rows in one dataset with one row in another dataset.

Before we proceed with the merge, we should make sure we have all of the data we want to unify. We mentioned the survey data earlier (`survey_all.txt` and `survey_d75.txt`), but we didn't read those files in because they're in a slightly more complex format. The files are *tab delimited* and encoded with `Windows-1252` encoding. An encoding defines how a computer stores the contents of a file in binary.

We'll need to specify the encoding and delimiter to the pandas [pandas.read_csv()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) function to ensure it reads the surveys in properly.

After we read in the survey data, we'll want to combine it into a single dataframe. We can do this by calling the [pandas.concat()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html).

In [5]:
# Read in 'survey_all.txt'
all_survey = pd.read_csv('schools/survey_all.txt', delimiter='\t', encoding='windows-1252')

# Read in 'survey_d75.txt'
d75_survey = pd.read_csv('schools/survey_d75.txt', delimiter='\t', encoding='windows-1252')

# Combine 'd75_survey' and 'all_survey' into a single dataframe
survey = pd.concat([all_survey, d75_survey], axis=0)

# Display the first five rows of 'survey'
survey.head()

,dbn,bn,schoolname,d75,studentssurveyed,highschool,schooltype,rr_s,rr_t,rr_p,...,s_q14_2,s_q14_3,s_q14_4,s_q14_5,s_q14_6,s_q14_7,s_q14_8,s_q14_9,s_q14_10,s_q14_11
0,01M015,M015,P.S. 015 Roberto Clemente,0,No,0.0,Elementary School,NaN,88,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01M019,M019,P.S. 019 Asher Levy,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01M020,M020,P.S. 020 Anna Silver,0,No,0.0,Elementary School,NaN,88,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01M034,M034,P.S. 034 Franklin D. Roosevelt,0,Yes,0.0,Elementary / Middle School,89.0,73,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01M063,M063,P.S. 063 William McKinley,0,No,0.0,Elementary School,NaN,100,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are two immediate facts that we can see in the output of `survey` dataframe:

- There are over `2000` columns, nearly all of which we don't need. We'll have to filter the data to remove the unnecessary ones. Working with fewer columns makes it easier to print the dataframe out and find correlations within it.
- The survey data has a `dbn` column that we'll want to convert to uppercase (`DBN`). The conversion makes the column name consistent with the other datasets.

## Data Cleaning

### 1. Filter Columns in survey dataset

We'll need to filter the columns to remove the ones we don't need. Luckily, there's a data dictionary at the [original data download location](https://data.cityofnewyork.us/Education/NYC-School-Survey-2011/mnz3-dyi8). The dictionary tells us what each column represents. Based on our knowledge of the problem and the analysis we're trying to do, we can use the data dictionary to determine which columns to use.

Here's a preview of the data dictionary:

<img src="https://s3.amazonaws.com/dq-content/xj5ud4r.png" width="800" height="800" />

*Source*: [Dataquest](https://www.dataquest.io/new-home/)

Based on the dictionary, it looks like these are the relevant columns:

```
["dbn", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
```

These columns give us aggregate survey data about how parents, teachers, and students feel about school safety, academic performance, and more. It also gives us the `DBN`, which allows us to uniquely identify the school.

Before we filter columns out, we'll want to copy the data from the `dbn` column into a new column called `DBN`.

In [6]:
# Copy 'survey' dataframe to a new dataframe to avoid 'PerformanceWarning'
survey2 = survey.copy()

# Copy the data from the 'dbn' column of 'survey' into new column called 'DBN'
survey2['DBN'] = survey2['dbn']

# Create the list of target columns
# note we want target the 'DBN' column instead of 'dbn' 
target_cols = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t",
               "N_p", "saf_p_11", "com_p_11", "eng_p_11",
               "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11",
               "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11",
               "aca_s_11", "saf_tot_11", "com_tot_11",
               "eng_tot_11", "aca_tot_11"]

# Filter 'survey2' so it only contains the columns listed above
survey2 = survey2.loc[:, target_cols]

# Assign 'survey2' dataframe to the key 'survey' in the dictionary 'data'
data['survey'] = survey2

# View the number of rows and columns in 'survey2' dataframe
data['survey'].shape

(1702, 23)

After filtering the columns, the `data['survey']` dataframe has `23` columns and `1702` columns.

## 2. Create "DBN" Column for class_size and hs_directory datasets

When we explored all of the datasets, we noticed that some of them, like `class_size` and `hs_directory`, don't have a `DBN` column. `hs_directory` does have a `dbn` column, though, so we can just rename it.

However, `class_size` doesn't appear to have the column at all. Here are the first few rows of the dataset:

In [7]:
data['class_size'].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN


And below are the first few rows of the `sat_results` data, which does have a `DBN` column:

In [8]:
data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384


From looking at these rows, we can tell that the `DBN` in the `sat_results` data is just a combination of the `CSD` and `SCHOOL CODE` columns in the `class_size` data. The main difference is that the `DBN` is padded, so that the `CSD` portion of it always consists of two digits. That means we'll need to add a leading `0` to the `CSD` if the `CSD` is less than two digits long. Here's a diagram illustrating what we need to do:

| CSD | Padded CSD |
| --- | :---: |
| 1 | 01 |
| 19 | 19 |
| 2 | 02 |
| 99 | 99 |

As we can see, whenever the `CSD` is less than two digits long, we need to add a leading `0`. We can accomplish this using the [pandas.Series.apply()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.apply.html) method, along with a common function that:

- Takes in a number.
- Converts the number to a string using the [str()](https://docs.python.org/3/library/stdtypes.html#str) function.
- Checks the length of the string using the [len()](https://docs.python.org/3/library/functions.html#len) function.
    - If the string is two digits long, returns the string.
    - If the string is one digit long, adds a `0` to the front of the string, then returns it.
        - We can use the string method [zfill()](https://docs.python.org/3/library/stdtypes.html#str.zfill) to do this.

Once we've padded the `CSD`, we can use the addition operator (`+`) to combine the values in the `CSD` and `SCHOOL CODE` columns. Here's a diagram illustrating the basic concept:

| Padded CSD |  | School Code |  | DBN |
| :---: | :---: | :---: | :---: | :---: |
| 01 |  | M015 |  | 01M015 |
| 19 | + | M022 | = | 19M022 |         
| 02 |  | M016 |  | 02M016 |
| 99 |  | M025 |  | 99M025 |

Before manipulating data in the `class_size` dataframe, first we'll copy the `dbn` column in `hs_directory` into a new column called `DBN`.

In [9]:
# Access 'data' directionary to copy column inthe 'hs_directory'
data['hs_directory']['DBN'] = data['hs_directory']['dbn']

# View the first five rows of 'hs_directory'
data['hs_directory'].head()

,dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority03,priority04,priority05,priority06,priority07,priority08,priority09,priority10,Location 1,DBN
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",17K548
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9,12,NaN,NaN,...,Then to Bronx students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",09X543
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6,12,NaN,NaN,...,Then to New York City residents who attend an ...,Then to Bronx students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",09X327
3,02M280,Manhattan Early College School for Advertising,Manhattan,M520,718-935-3477,NaN,9,10,9,14.0,...,Then to Manhattan students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",02M280
4,28Q680,Queens Gateway to Health Sciences Secondary Sc...,Queens,Q695,718-969-3155,718-969-3552,6,12,NaN,NaN,...,Then to Queens students or residents,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,NaN,"160-20 Goethals Avenue\nJamaica, NY 11432\n(40...",28Q680


Next we'll create a new column called `padded_csd` in the `class_size` dataset. We'll use `pandas.Series.apply()` method along with a custom function `check_num()` to generate this column.

In [10]:
def check_num(num):
    """
    Convert the integer number into string then add a zero (0) in the beginning of the number
    if the length is less than two.
    
    Param (int):
        The number can have one or more digits.
    
    Returns (str):
        The returned number has two digits atleast.
    """
    return str(num).zfill(2)

# Create new column 'padded_csd' in 'class_size'
data['class_size']['padded_csd'] = data['class_size']['CSD'].apply(check_num)

# Add 'padded_csd' and 'SCHOOL CODE' to create 'DBN' column
data['class_size']['DBN'] = data['class_size']['padded_csd'] + data['class_size']['SCHOOL CODE']

# Display first five rows of 'class_size' dataframe
data['class_size'].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01,01M015
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01,01M015
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01,01M015


### 3. Add SAT Columns in sat_results dataset

Now, let's take some time to calculate variable that are useful in our analysis. We've already discussed one such variable -- a column that totals up the SAT scores for the different sections of the exam. This makes it much easier to correlate scores with demographic factors because we'll be working with a single number, rather than three different ones.

Before we can generate this column, we'll need to convert the `SAT Math Avg. Score`, `SAT Critical Reading Avg. Score`, and `SAT Writing Avg. Score` columns in the `sat_results` dataset from the object (string) data type to a numeric data type. We can use [pandas.to_numeric()](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.to_numeric.html) method for the conversion. If we don't convert the values, we won't be able to add the columns together.

It's important to pass the keyword argument `errors="coerce"` when we call `pandas.to_numeric()`, so that pandas treats any invalid strings it can't convert to numbers an missing values instead.

After we perform the conversion, we can use [pandas.DataFrame.sum()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) to add all three columns together.

In [11]:
# Create a list of target columns
target_cols = ['SAT Math Avg. Score',
               'SAT Critical Reading Avg. Score',
               'SAT Writing Avg. Score']

# Convert the data type to numeric of the target columns
# and assign the result back to the same column
for col in target_cols:
    data['sat_results'][col] = pd.to_numeric(data['sat_results'][col], errors='coerce')
    
# Create a column 'sat_score' in 'sat_results'
# that holds the combined SAT score
data['sat_results']['sat_score'] = data['sat_results'][target_cols].sum(axis=1)

# Display first five rows
data['sat_results'].head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,1149.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,1174.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,1207.0


### 4. Extract Coordinates in hs_directory

Now we'll want to parse the latitude and longitude coordinates for each school. This enables us to map the schools and uncover any geographic patterns in the data. The coordinates are currently in the text field `Location 1` in the `hs_directory` dataset.

Let's take a look at the first few rows:

In [12]:
data['hs_directory']['Location 1'].head()

0    883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...
1    1110 Boston Road\nBronx, NY 10456\n(40.8276026...
2    1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...
3    411 Pearl Street\nNew York, NY 10038\n(40.7106...
4    160-20 Goethals Avenue\nJamaica, NY 11432\n(40...
Name: Location 1, dtype: object

As we can see, this field contains a lot of information we don't need. We want to extract the coordinates, which are in parentheses at the end of the field. Here's an example:

In [13]:
data['hs_directory']['Location 1'].iloc[1]

'1110 Boston Road\nBronx, NY 10456\n(40.8276026690005, -73.90447525699966)'

We want to extract the latitude, `40.8276026690005`, and the longitude, `-73.90447525699966`. Taken together, latitude and longitude make up a pair of coordinates that allows us to pinpoint any location on Earth.

We can do the extraction with a regular expression. This following expression pulls out everything inside the parentheses:

In [14]:
# Extract coordinates at first index of 'Lacation 1'
data['hs_directory']['Location 1'].str.findall("\(.+\)")[0]

['(40.67029890700047, -73.96164787599963)']

This command returns a list of coordinates. We'll need to process this result further using the string methods [split()](https://docs.python.org/3/library/stdtypes.html#str.split) and [replace()](https://docs.python.org/3/library/stdtypes.html#str.replace) methods to extract each coordinate.

In [18]:
import re
def extract_coords(field_info, indx, pat=r'\(.+\)'):
    """
    Extract the coordinates for each school from
    'Location 1' column and split them into latitude and longitude.
    
    Params:
        field_info (str): The information of school, e.g. address, coordinates, etc
        indx (int): Index number to extract latitude and longitude
        pat (regex): Regular Expression to extract everything inside the parentheses of 'field_info'
        
    Return:
        If the indx is 0 return latitude values otherwise return the longitude
    """
    if indx == 0:
        return re.findall(pat, field_info)[0].split(',')[indx].replace('(', '')
    elif indx == 1:
        return re.findall(pat, field_info)[0].split(',')[indx].replace(')', '').strip()
    
# Extract latitude and assign values to new column 'lat' of 'hs_directory'
data['hs_directory']['lat'] = data['hs_directory']['Location 1'].apply(extract_coords, indx=0)

# Extract longitude and assign values to new column 'lon' of 'hs_directory'
data['hs_directory']['lon'] = data['hs_directory']['Location 1'].apply(extract_coords, indx=1)

# Display the result
data['hs_directory'].head()

,dbn,school_name,boro,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,priority05,priority06,priority07,priority08,priority09,priority10,Location 1,DBN,lat,lon
0,17K548,Brooklyn School for Music & Theatre,Brooklyn,K440,718-230-6250,718-230-6262,9,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...",17K548,40.67029890700047,-73.96164787599963
1,09X543,High School for Violin and Dance,Bronx,X400,718-842-0687,718-589-9849,9,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"1110 Boston Road\nBronx, NY 10456\n(40.8276026...",09X543,40.8276026690005,-73.90447525699966
2,09X327,Comprehensive Model School Project M.S. 327,Bronx,X240,718-294-8111,718-294-8109,6,12,NaN,NaN,...,Then to New York City residents,NaN,NaN,NaN,NaN,NaN,"1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...",09X327,40.842414068000494,-73.91616158599965
3,02M280,Manhattan Early College School for Advertising,Manhattan,M520,718-935-3477,NaN,9,10,9,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"411 Pearl Street\nNew York, NY 10038\n(40.7106...",02M280,40.71067947100045,-74.00080702099967
4,28Q680,Queens Gateway to Health Sciences Secondary Sc...,Queens,Q695,718-969-3155,718-969-3552,6,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"160-20 Goethals Avenue\nJamaica, NY 11432\n(40...",28Q680,40.718810094000446,-73.80650045499965


We have the both coordinates, `lat` and `lon`. Next, we'll need to convert them to numeric values. We can use the `pandas.to_numeric()` function to convert them from strings to numbers.

In [27]:
# List of the target columns
target_cols = ['lat', 'lon']

for col in target_cols:
    data['hs_directory'][col] = pd.to_numeric(data['hs_directory'][col], errors='coerce')
    
# Check the data type of 'lat' and 'lon' column (should be float type)
data['hs_directory'][['lat', 'lon']].dtypes

lat    float64
lon    float64
dtype: object

### 5. Make Unique "DBN" Value in Each Dataset

So far we have manipulated several datasets and found that we could combine them using the `DBN` column. All of the datasets are currently stored as keys in the `data` dictionary where each dataset is a pandas dataframe.

However, we'll need to do some more data cleaning before combining these datasets. Then, we'll compute correlations and perform some analysis.

The first thing we'll need to do in preparation for the merge is condense some of the datasets. In the last lesson, we noticed that the values in the `DBN` column were unique in the `sat_results` dataset. Other datasets like `class_size` had duplicate `DBN` values.

We'll need to condense these datasets so that each value in the `DBN` column is unique. If not, we'll run into issues when it comes time to combine the datasets.

While the main dataset we want to analyze, `sat_results`, has unique `DBN` values for every high school in New York City, other datasets aren't as clean. A single row in the `sat_results` dataset may match multiple rows in the `clean_size` dataset, for example. This situation creates problems, because we don't know which of the multiple entries in the `clean_size` dataset we should combine with the single matchin entry in `sat_results`. Below is a table that illustrates the problem:

| sat_results |  |  | class_size |  |
| :---: | :---: | :---: | :---: | :---: |
| **DBN** | ... |  | **DBN** | ... |
| 01M022 | ... |  | 01M022 | ... |
| 05M345 | ... | + | 01M022 | ... |
| 02M456 | ... |  | 05M345 | ... |
| 99M520 | ... |  | 05M345 | ... |

In the table above, we can't combine the rows from both datasets, because there are several cases where multiple rows in `class_size` match a single row in `sat_results`.

To resolve this issue, we'll condense the `class_size`, `graduation` and `demographics` datasets so that each `DBN` is unique.

### 5.1 class_size dataset

The first dataset that we'll condense is `class_size`. The first few rows of the `class_size` look like this:

In [28]:
data['class_size'].head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
0,1,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01,01M015
1,1,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01,01M015
2,1,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
3,1,M,M015,P.S. 015 Roberto Clemente,01,CTT,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01,01M015
4,1,M,M015,P.S. 015 Roberto Clemente,02,GEN ED,-,-,-,15.0,1.0,15.0,15.0,15.0,ATS,NaN,01,01M015


As we can see, the first few rows all pertain to the same school, which is why the `DBN` appears more than once. It looks like each school has multiple values for `GRADE`, `PROGRAM TYPE`, `CORE SUBJECT (MS CORE and 9-12 ONLY)`, and `CORE COURSE (MS CORE and 9-12 ONLY)`.

If we look at the unique values for `GRADE`, we get the following:

In [32]:
data['class_size']['GRADE '].unique()

array(['0K', '01', '02', '03', '04', '05', '0K-09', nan, '06', '07', '08',
       'MS Core', '09-12', '09'], dtype=object)

Since we're dealing with high schools, we're only concerned with grades `9` through `12`. That means we only want to pick rows where the value in the `GRADE` column is `09-12`.

If we look at the unique values for `PROGRAM TYPE`, we get the following:

In [33]:
data['class_size']['PROGRAM TYPE'].unique()

array(['GEN ED', 'CTT', 'SPEC ED', nan, 'G&T'], dtype=object)

Each school can have multiple program types. Since `GEN ED` is the largest category by far, let's only select rows where `PROGRAM TYPE` is `GEN ED`.

In [42]:
# Create a new variable 'class_size' and assign the value of
# data['class_size'] to it
class_size = data['class_size']

# Filter 'class_size' so the 'GRADE' column only contains the value '09-12'
# also filter the dataset where the values are 'GEN ED' in 'PROGRAM TYPE' column
class_size = (class_size[(class_size['GRADE '] == '09-12') &
            (class_size['PROGRAM TYPE'] == 'GEN ED')])

# Display first five rows
class_size.head()

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,padded_csd,DBN
225,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 9,-,63.0,3.0,21.0,19.0,25.0,STARS,NaN,01,01M292
226,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 10,-,79.0,3.0,26.3,24.0,31.0,STARS,NaN,01,01M292
227,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 11,-,38.0,2.0,19.0,16.0,22.0,STARS,NaN,01,01M292
228,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 12,-,69.0,3.0,23.0,13.0,30.0,STARS,NaN,01,01M292
229,1,M,M292,Henry Street School for International Studies,09-12,GEN ED,MATH,Integrated Algebra,-,53.0,3.0,17.7,16.0,21.0,STARS,NaN,01,01M292


As we can see the result of `class_size` above, the `DBN` column still isn't completely unique. This is due to the `CORE COURSE (MS CORE and 9-12 ONLY)` and `CORE SUBJECT (MS CORE and 9-12 ONLY)` columns. These columns seem to pertain to different kinds of classes. For example, here are the unique values for `CORE SUBJECT (MS CORE and 9-12 ONLY)`:

In [45]:
class_size['CORE SUBJECT (MS CORE and 9-12 ONLY)'].unique()

array(['ENGLISH', 'MATH', 'SCIENCE', 'SOCIAL STUDIES'], dtype=object)

This column only seems to include certain subjects. We want our class size data to include every single class a school offers -- not just a subset of them. What we can do is take the average across all of the classes a school offers. This gives us unique `DBN` values, while also incorporating as much data as possible into the average.

Fortunately, we can use the [pandas.DataFrame.groupby()](http://pandas.pydata.org/pandas-docs/stable/groupby.html) method to help us with this. The `DataFrame.groupby()` method splits a dataframe up into unique groups, based on a given column. We can then use the [agg()](http://pandas.pydata.org/pandas-docs/stable/groupby.html#aggregation) method on the resulting `pandas.core.groupby` object to find the mean of each column.

Let's say we have this dataset: